In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import itertools

import pandas as pd
import rasterio
import utm
from IPython.display import Image
import folium
import utils

import ee
# ee.Authenticate()
ee.Initialize()

In [ ]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

In [ ]:
# Initial date of interest (inclusive).
i_date = '2021-08-17'

# Final date of interest (exclusive).
f_date = '2021-08-19'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [ ]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

In [ ]:
scale = 1000  # scale in meters

# Print the elevation near Lyon, France.
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

In [ ]:
# Get the data for the pixel intersecting the point in urban area.
lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# Get the data for the pixel intersecting the point in rural area.
lst_r_poi = lst.getRegion(r_poi, scale).getInfo()

# Preview the result.
lst_u_poi[:5]

In [ ]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [ ]:
lst_df_urban = ee_array_to_df(lst_u_poi, ['LST_Day_1km'])

def t_modis_to_celsius(t_modis):
    """Converts MODIS LST units to degrees Celsius."""
    return 0.02 * t_modis - 273.15 # °C

# Apply the function to get temperature in celsius.
lst_df_urban['LST_Day_1km'] = lst_df_urban['LST_Day_1km'].apply(t_modis_to_celsius)

# Do the same for the rural point.
lst_df_rural = ee_array_to_df(lst_r_poi, ['LST_Day_1km'])
lst_df_rural['LST_Day_1km'] = lst_df_rural['LST_Day_1km'].apply(t_modis_to_celsius)

lst_df_urban.head()

In [ ]:
# Fitting curves.
## First, extract x values (times) from the dfs.
x_data_u = np.asanyarray(lst_df_urban['time'].apply(float))  # urban
x_data_r = np.asanyarray(lst_df_rural['time'].apply(float))  # rural

## Secondly, extract y values (LST) from the dfs.
y_data_u = np.asanyarray(lst_df_urban['LST_Day_1km'].apply(float))  # urban
y_data_r = np.asanyarray(lst_df_rural['LST_Day_1km'].apply(float))  # rural

## Then, define the fitting function with parameters.
def fit_func(t, lst0, delta_lst, tau, phi):
    return lst0 + (delta_lst / 2) * np.sin(2 * np.pi * t / tau + phi)

## Optimize the parameters using a good start p0.
lst0 = 20
delta_lst = 40
tau = 365 * 24 * 3600 * 1000   # milliseconds in a year
phi = 2 * np.pi * 4 * 30.5 * 3600 * 1000 / tau  # offset regarding when we expect LST(t)=LST0

params_u, params_covariance_u = optimize.curve_fit(
    fit_func, x_data_u, y_data_u, p0=[lst0, delta_lst, tau, phi])
params_r, params_covariance_r = optimize.curve_fit(
    fit_func, x_data_r, y_data_r, p0=[lst0, delta_lst, tau, phi])

# Subplots.
fig, ax = plt.subplots(figsize=(14, 6))

# Add scatter plots.
ax.scatter(lst_df_urban['datetime'], lst_df_urban['LST_Day_1km'],
           c='black', alpha=0.2, label='Urban (data)')
ax.scatter(lst_df_rural['datetime'], lst_df_rural['LST_Day_1km'],
           c='green', alpha=0.35, label='Rural (data)')

# Add fitting curves.
ax.plot(lst_df_urban['datetime'],
        fit_func(x_data_u, params_u[0], params_u[1], params_u[2], params_u[3]),
        label='Urban (fitted)', color='black', lw=2.5)
ax.plot(lst_df_rural['datetime'],
        fit_func(x_data_r, params_r[0], params_r[1], params_r[2], params_r[3]),
        label='Rural (fitted)', color='green', lw=2.5)

# Add some parameters.
ax.set_title('Daytime Land Surface Temperature Near Lyon', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Temperature [C]', fontsize=14)
ax.set_ylim(-0, 40)
ax.grid(lw=0.2)
ax.legend(fontsize=14, loc='lower right')

plt.show()

In [ ]:
# Define a region of interest with a buffer zone of 1000 km around Lyon.
roi = u_poi.buffer(1e6)

In [ ]:
# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

In [ ]:
# Create a URL to the styled image for a region around France.
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

In [ ]:
# Make pixels with elevation below sea level transparent.
elv_img = elv.updateMask(elv.gt(0))

# Display the thumbnail of styled elevation in France.
Image(url=elv_img.getThumbURL({
    'min': 0, 'max': 2000, 'dimensions': 512, 'region': roi,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [ ]:
# Create a buffer zone of 10 km around Lyon.
lyon = u_poi.buffer(10000)  # meters

url = elv_img.getThumbUrl({
    'min': 150, 'max': 350, 'region': lyon, 'dimensions': 512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

In [ ]:
# Get a feature collection of administrative boundaries.
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

# Filter the feature collection to subset France.
france = countries.filter(ee.Filter.eq('ADM0_NAME', 'France'))

# Clip the image by France.
elv_fr = elv_img.clip(france)

# Create the URL associated with the styled image data.
url = elv_fr.getThumbUrl({
    'min': 0, 'max': 2500, 'region': roi, 'dimensions': 512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display a thumbnail of elevation in France.
Image(url=url)

In [ ]:
# Define the center of our map.
lat, lon = 45.77, 4.855

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Select a specific band and dates for land cover.
lc_img = lc.select('LC_Type1').filterDate(i_date).first()

# Set visualization parameters for land cover.
lc_vis_params = {
    'min': 1,'max': 17,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

# Create a map.
lat, lon = 45.77, 4.855
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(lc_img, lc_vis_params, 'Land Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
my_map

In [ ]:
my_map.save('my_lc_interactive_map.html')

In [ ]:
# Set visualization parameters for ground elevation.
elv_vis_params = {
    'min': 0, 'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Set visualization parameters for land surface temperature.
lst_vis_params = {
    'min': 0, 'max': 40,
    'palette': ['white', 'blue', 'green', 'yellow', 'orange', 'red']}

# Arrange layers inside a list (elevation, LST and land cover).
ee_tiles = [elv_img, lst_img, lc_img]

# Arrange visualization parameters inside a list.
ee_vis_params = [elv_vis_params, lst_vis_params, lc_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['Elevation', 'Land Surface Temperature', 'Land Cover']

# Create a new map.
lat, lon = 45.77, 4.855
my_map = folium.Map(location=[lat, lon], zoom_start=5)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

my_map

# Test Var fire

In [ ]:
arr1 = rasterio.open('./output/before_var.tiff').read(1)
arr2 = rasterio.open('./output/after_var.tiff').read(1)
diff = arr1 - arr2

In [ ]:
v1 = 6800
v2 = 9000
delta_v = v2 - v1
h1 = 8780
h2 = h1 + delta_v

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(diff)
plt.vlines(v1, 0, diff.shape[0], color='r', linestyle='dashed', linewidth=1)
plt.vlines(v2, 0, diff.shape[0], color='r', linestyle='dashed', linewidth=1)
plt.hlines(h1, 0, diff.shape[1], color='r', linestyle='dashed', linewidth=1)
plt.hlines(h2, 0, diff.shape[1], color='r', linestyle='dashed', linewidth=1)
# plt.grid(True, which='both')
plt.tight_layout()
plt.show()

In [ ]:
fire = diff[h1:h2, v1:v2]
plt.imshow(fire)
plt.show()

In [ ]:
from skimage.morphology import square, closing

tt = utils.threshold_filter(fire, 0.3)
plt.imshow(tt)
plt.show()

print(utils.calculate_area(tt, diff) * 100)

closed = closing(tt, square(25))
plt.imshow(closed)
plt.show()

print(utils.calculate_area(closed, diff) * 100)

In [ ]:
# from the pixels that are not null, we take a random sample (1%)
# so we can then transform the pixels into coordinates
# this guarantees than (in theory) the shape of the fire is kept
np.random.seed(42)
randoms = np.zeros_like(closed)
for i in range(closed.shape[0]):
    for j in range(closed.shape[1]):
        if closed[i, j] != 0.:
            randoms[i, j] = np.random.choice([1, 0], p=[0.01, 0.99])

In [ ]:
# x, y = diff.shape[0] / 2, diff.shape[1] / 2 # center of the image
x, y = 1500, 1500
plt.imshow(randoms)
plt.plot(x, y, 'ro')
plt.show()

In [ ]:
xx, yy = v1 + x, h1 + y # shift the coordinates

plt.imshow(diff)
plt.plot(xx, yy, 'ro')
plt.show()

In [ ]:
image_folder = 'data/S2B_MSIL2A_20210802T102559_N0301_R108_T31TGJ_20210802T133728.SAFE/'
tci_file_path = utils.get_tci_file_path(image_folder)

In [ ]:
# center of the fire image
pixel_column = int(xx)
pixel_row = int(yy)
print(f"pixel_column: {pixel_column}")
print(f"pixel_row: {pixel_row}")

transform = rasterio.open(tci_file_path, driver='JP2OpenJPEG').transform
zone_number = int(tci_file_path.split("/")[-1][1:3])
zone_letter = tci_file_path.split("/")[-1][0]

utm_x, utm_y = transform[2], transform[5]

# Converting pixel position to UTM
east = utm_x + pixel_column * 10
north = utm_y + pixel_row * - 10

# Converting UTM to latitude and longitude
latitude, longitude = utm.to_latlon(east, north, zone_number, zone_letter)
print(f"latitude, longitude: {latitude}, {longitude}")

# Converting latitude and longitude to UTM
east, north, zone_number, zone_letter = utm.from_latlon(latitude, longitude, force_zone_number=zone_number)

# Converting UTM to column and row
pixel_column = round((east - utm_x) / 10)
pixel_row = round((north - utm_y) / - 10)
print(f"pixel_column: {pixel_column}")
print(f"pixel_row: {pixel_row}")

The coordinates correspond to ~center of the fire

TODO:

* Take every pixel of `fire` and transform them into coordinates
* Create a shapefile from those coordinates
* Also retrieve soil type from these coordinates

In [ ]:
def get_coordinates_from_pixels(img, h, v, img_folder):
    coords = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i, j] != 0.:
                coords.append((h + i, v + j))

    tci_file_path = utils.get_tci_file_path(img_folder)
    transform = rasterio.open(tci_file_path, driver='JP2OpenJPEG').transform
    zone_number = int(tci_file_path.split("/")[-1][1:3])
    zone_letter = tci_file_path.split("/")[-1][0]
    utm_x, utm_y = transform[2], transform[5]
    coords_utm = []
    for coord in coords:
        x, y = coord
        east = utm_x + x * 10
        north = utm_y + y * - 10
        latitude, longitude = utm.to_latlon(east, north, zone_number, zone_letter)
        coords_utm.append((latitude, longitude))
    return coords_utm

In [ ]:
coords_utm = get_coordinates_from_pixels(randoms, h1, v1, image_folder)
coords_utm[:5]

In [ ]:
coords_utm = pd.DataFrame(coords_utm, columns=['latitude', 'longitude'])
coords_utm.to_csv('data/coords_utm_var.csv', index=False)
coords_utm.head()

In [ ]:
lc_type = pd.read_csv('data/MODIS_LandCover_Type1.csv', sep=',', header=0)
lc_type['Color'] = lc_type['Color'].apply(lambda x: f'#{x}')
lc_type

In [ ]:
import time
from collections import Counter
import matplotlib.colors as mplcols

In [ ]:
np.random.seed(42)
scale = 1000
covers = []
choose = 500
random_idxs = np.sort(np.random.choice(range(len(coords_utm)), size=choose, replace=False))

In [ ]:
for i in random_idxs:
# for i in range(choose):
    u_lat, u_lon = coords_utm.iloc[i]['latitude'], coords_utm.iloc[i]['longitude']
    u_poi = ee.Geometry.Point(u_lon, u_lat)
    time.sleep(0.25)

    try:
        lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
        covers.append(lc_urban_point)
    except:
        print('error with earth engine')

In [ ]:
if None in covers:
    covers = [i for i in covers if i] # remove None values
covers = dict(Counter(covers))
covers

In [ ]:
sum(covers.values())

In [ ]:
labels = [lc_type.loc[lc_type['Value'] == i]['Description'].values[0] for i in covers.keys()]
labels = [i.split(':')[0] for i in labels]
labels

In [ ]:
colors = [lc_type.loc[lc_type['Value'] == i]['Color'].values[0] for i in covers.keys()]
colors = [mplcols.to_rgb(i) for i in colors]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(aspect="equal"))
wedges, texts, autotexts = ax.pie(covers.values(),
    autopct='%1.1f%%', startangle=90,
    textprops=dict(color='w'))
    # colors=colors))
ax.legend(wedges, labels,
    title='Land Cover Type',
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=8, weight="bold")
plt.show()

<img src="output/pie.gif" width="750" align="center">

In [ ]:
# Afficher image RGB
# Graphe des couvertures en fonction du nombre de points